# Prototype Pattern

## Intent
Specify the kinds of objects to create using a prototypical instance, and create new objects by copying this prototype.

## Problem
You need to create objects but:
- Creation is expensive (database queries, computations)
- Object initialization is complex
- Want to avoid subclass explosion
- Need to create objects at runtime without knowing exact classes

**Real-world analogy**: Cell division - new cells created by copying existing cell

## When to Use
✅ **Use when:**
- Object creation is expensive
- Objects have only few state combinations
- Want to avoid factory hierarchies
- Objects are similar with slight variations
- Need to create objects at runtime

❌ **Avoid when:**
- Objects are simple to create from scratch
- Deep copying is problematic
- Objects have circular references

## Pattern Structure
```
┌───────────┐
│ Prototype │
├───────────┤
│ clone()   │
└───────────┘
      ▲
      │
┌─────┴──────┐
│Concrete    │
│Prototype   │
├────────────┤
│clone()     │
└────────────┘
```

## Python's Built-in Support

Python provides `copy` module:
- `copy.copy()`: Shallow copy
- `copy.deepcopy()`: Deep copy

## Example 1: Without Prototype

**Problem**: Expensive object creation

In [ ]:
import time

# WITHOUT Prototype - Recreate everything from scratch
class ExpensiveObject:
    def __init__(self, data_type):
        print(f"Creating {data_type} object...")
        time.sleep(0.5)  # Simulate expensive operation
        self.data = self._load_data(data_type)
        print(f"  Loaded {len(self.data)} items")
    
    def _load_data(self, data_type):
        # Simulate expensive database query or file I/O
        return [f"{data_type}_item_{i}" for i in range(1000)]

# Creating similar objects is slow!
print("Creating 3 similar objects:")
start = time.time()
obj1 = ExpensiveObject("UserData")
obj2 = ExpensiveObject("UserData")
obj3 = ExpensiveObject("UserData")
print(f"\nTime: {time.time() - start:.2f}s")
print("\n❌ Slow! Each object loads same data from scratch!")

## Implementation: Prototype Pattern (Using Python's copy)

In [ ]:
import copy
from abc import ABC, abstractmethod
import time

# Prototype interface
class Prototype(ABC):
    @abstractmethod
    def clone(self):
        pass


# Concrete prototype
class DataObject(Prototype):
    def __init__(self, data_type: str):
        print(f"  🏗️  Creating {data_type} object (expensive!)...")
        time.sleep(0.5)  # Expensive operation
        self.data_type = data_type
        self.data = self._load_data(data_type)
        self.settings = {"version": 1, "cached": True}
        print(f"  ✅ Loaded {len(self.data)} items")
    
    def _load_data(self, data_type: str) -> list:
        return [f"{data_type}_item_{i}" for i in range(1000)]
    
    def clone(self) -> 'DataObject':
        """Clone using shallow copy."""
        print(f"  ⚡ Cloning {self.data_type} (fast!)")
        return copy.copy(self)
    
    def deep_clone(self) -> 'DataObject':
        """Clone using deep copy."""
        print(f"  🔄 Deep cloning {self.data_type}")
        return copy.deepcopy(self)


# Demo
print("\n=== Prototype Pattern ===")

print("\n1. Creating original object (slow):")
start = time.time()
original = DataObject("UserData")
print(f"   Time: {time.time() - start:.2f}s")

print("\n2. Cloning objects (fast):")
start = time.time()
clone1 = original.clone()
clone2 = original.clone()
clone3 = original.clone()
print(f"   Time: {time.time() - start:.2f}s")

print("\n3. Verifying clones:")
print(f"   Original data items: {len(original.data)}")
print(f"   Clone1 data items: {len(clone1.data)}")
print(f"   Original is Clone1: {original is clone1}")
print(f"   Original.data is Clone1.data: {original.data is clone1.data}")

print("\n✅ Cloning is much faster than creating from scratch!")

## Shallow vs Deep Copy

In [ ]:
import copy

# Object with nested structure
class Person:
    def __init__(self, name: str, age: int, address: dict):
        self.name = name
        self.age = age
        self.address = address  # Nested object
    
    def __repr__(self):
        return f"Person({self.name}, {self.age}, {self.address})"


# Demo
print("\n=== Shallow vs Deep Copy ===")

original = Person("Alice", 30, {"city": "NYC", "zip": "10001"})
print(f"\nOriginal: {original}")

# Shallow copy
print("\n1. Shallow copy (copy.copy):")
shallow = copy.copy(original)
print(f"   Shallow: {shallow}")
print(f"   Original is Shallow: {original is shallow}")
print(f"   Original.address is Shallow.address: {original.address is shallow.address}")

# Modify shallow copy's address
shallow.address["city"] = "LA"
print(f"\n   After modifying shallow.address['city']:")
print(f"   Original.address: {original.address}")
print(f"   Shallow.address: {shallow.address}")
print("   ⚠️  Original affected! Address is shared!")

# Deep copy
print("\n2. Deep copy (copy.deepcopy):")
original2 = Person("Bob", 25, {"city": "NYC", "zip": "10001"})
deep = copy.deepcopy(original2)
print(f"   Deep: {deep}")
print(f"   Original is Deep: {original2 is deep}")
print(f"   Original.address is Deep.address: {original2.address is deep.address}")

# Modify deep copy's address
deep.address["city"] = "SF"
print(f"\n   After modifying deep.address['city']:")
print(f"   Original.address: {original2.address}")
print(f"   Deep.address: {deep.address}")
print("   ✅ Original unaffected! Address is independent!")

## Real-World Example: Game Characters

In [ ]:
import copy
from typing import List

# Character prototype
class GameCharacter:
    """Game character with equipment and skills."""
    
    def __init__(self, name: str, char_class: str):
        self.name = name
        self.char_class = char_class
        self.level = 1
        self.health = 100
        self.equipment = []
        self.skills = []
        self._load_class_data(char_class)
    
    def _load_class_data(self, char_class: str):
        """Load class-specific data (expensive operation)."""
        if char_class == "Warrior":
            self.equipment = ["Sword", "Shield", "Armor"]
            self.skills = ["Slash", "Block", "Charge"]
        elif char_class == "Mage":
            self.equipment = ["Staff", "Robe"]
            self.skills = ["Fireball", "Ice Blast", "Teleport"]
        elif char_class == "Archer":
            self.equipment = ["Bow", "Quiver", "Leather Armor"]
            self.skills = ["Arrow Shot", "Rapid Fire", "Eagle Eye"]
    
    def clone(self) -> 'GameCharacter':
        """Clone character with deep copy."""
        return copy.deepcopy(self)
    
    def __repr__(self):
        return f"{self.name} ({self.char_class} Lv{self.level})"


# Character registry (prototype manager)
class CharacterRegistry:
    """Registry of character prototypes."""
    
    def __init__(self):
        self._prototypes = {}
    
    def register(self, key: str, prototype: GameCharacter) -> None:
        """Register a prototype."""
        self._prototypes[key] = prototype
    
    def create(self, key: str, name: str) -> GameCharacter:
        """Create character from prototype."""
        if key not in self._prototypes:
            raise ValueError(f"Unknown prototype: {key}")
        
        character = self._prototypes[key].clone()
        character.name = name
        return character


# Demo
print("\n=== Game Character Prototypes ===")

# Create registry with prototypes
registry = CharacterRegistry()

print("\n1. Creating prototypes:")
warrior_proto = GameCharacter("Warrior Prototype", "Warrior")
mage_proto = GameCharacter("Mage Prototype", "Mage")
archer_proto = GameCharacter("Archer Prototype", "Archer")

registry.register("warrior", warrior_proto)
registry.register("mage", mage_proto)
registry.register("archer", archer_proto)

print("   Registered 3 prototypes")

print("\n2. Creating characters from prototypes:")
player1 = registry.create("warrior", "Conan")
player2 = registry.create("mage", "Gandalf")
player3 = registry.create("warrior", "Aragorn")
player4 = registry.create("archer", "Legolas")

print(f"   {player1}")
print(f"     Equipment: {player1.equipment}")
print(f"     Skills: {player1.skills}")

print(f"   {player2}")
print(f"     Equipment: {player2.equipment}")
print(f"     Skills: {player2.skills}")

print("\n3. Customizing cloned characters:")
player1.level = 10
player1.equipment.append("Magic Sword")

print(f"   {player1} - Equipment: {player1.equipment}")
print(f"   {player3} - Equipment: {player3.equipment}")
print("   ✅ Independent copies!")

print("\n✅ Create characters quickly from prototypes!")

## Real-World Example: Document Templates

In [ ]:
import copy
from datetime import datetime

# Document prototype
class Document:
    """Document with template support."""
    
    def __init__(self, title: str, doc_type: str):
        self.title = title
        self.doc_type = doc_type
        self.created_at = datetime.now()
        self.sections = []
        self.metadata = {}
        self._load_template(doc_type)
    
    def _load_template(self, doc_type: str):
        """Load template structure."""
        if doc_type == "Report":
            self.sections = [
                {"title": "Executive Summary", "content": ""},
                {"title": "Introduction", "content": ""},
                {"title": "Analysis", "content": ""},
                {"title": "Conclusion", "content": ""}
            ]
            self.metadata = {"format": "PDF", "confidential": False}
        elif doc_type == "Invoice":
            self.sections = [
                {"title": "Billing Info", "content": ""},
                {"title": "Items", "content": ""},
                {"title": "Total", "content": ""}
            ]
            self.metadata = {"format": "PDF", "currency": "USD"}
    
    def clone(self) -> 'Document':
        """Clone document."""
        return copy.deepcopy(self)
    
    def display(self):
        print(f"\n📄 {self.title} ({self.doc_type})")
        print(f"   Created: {self.created_at.strftime('%Y-%m-%d %H:%M')}")
        print(f"   Sections: {len(self.sections)}")
        for section in self.sections:
            status = "✅" if section["content"] else "⬜"
            print(f"     {status} {section['title']}")


# Demo
print("\n=== Document Templates ===")

print("\n1. Creating templates:")
report_template = Document("Report Template", "Report")
invoice_template = Document("Invoice Template", "Invoice")

print("\n2. Creating documents from templates:")
q1_report = report_template.clone()
q1_report.title = "Q1 2024 Report"
q1_report.sections[0]["content"] = "Q1 was successful..."
q1_report.sections[1]["content"] = "In Q1, we..."

q2_report = report_template.clone()
q2_report.title = "Q2 2024 Report"
q2_report.sections[0]["content"] = "Q2 exceeded expectations..."

invoice1 = invoice_template.clone()
invoice1.title = "Invoice #001"
invoice1.sections[1]["content"] = "Consulting: $5000"
invoice1.sections[2]["content"] = "Total: $5000"

print("\n3. Displaying documents:")
q1_report.display()
q2_report.display()
invoice1.display()

print("\n✅ Quickly create documents from templates!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Performance**: Faster than creating from scratch
2. **Flexibility**: Add/remove prototypes at runtime
3. **Reduces subclassing**: Fewer factory classes needed
4. **Dynamic configuration**: Create objects with specific states
5. **Simplifies creation**: Complex initialization done once

### ❌ Disadvantages
1. **Cloning complexity**: Deep copy can be tricky
2. **Circular references**: Can cause issues
3. **Hidden dependencies**: May not be obvious what's being copied
4. **Mutable state**: Need to decide shallow vs deep copy

## Shallow vs Deep Copy

**Shallow Copy**:
- Copies object structure
- References point to same nested objects
- Fast
- Use when nested objects are immutable

**Deep Copy**:
- Copies entire object tree
- Independent nested objects
- Slower
- Use when objects contain mutable nested objects

## Common Use Cases

1. **Games**: Cloning characters, items, enemies
2. **Documents**: Templates, forms
3. **Configuration**: Settings with variations
4. **UI**: Widget templates
5. **Testing**: Test data creation
6. **Caching**: Cached objects as prototypes

## Related Patterns

- **Abstract Factory**: Can store prototypes
- **Composite**: Often paired with Prototype
- **Decorator**: Can be used with cloned objects
- **Memento**: Similar cloning concept

## Best Practices

1. **Use deep copy carefully**: Be aware of performance
2. **Implement clone method**: Provide clear cloning interface
3. **Handle circular references**: Use registry or special handling
4. **Document copy depth**: Clarify shallow vs deep
5. **Consider immutability**: Immutable objects don't need deep copy
6. **Registry pattern**: Manage prototypes centrally
7. **Reset state**: Clear any state that shouldn't be cloned

## Python-Specific Implementation

Python makes Prototype pattern easy with `copy` module:

```python
import copy

class MyClass:
    def clone(self):
        return copy.copy(self)  # Shallow
    
    def deep_clone(self):
        return copy.deepcopy(self)  # Deep
```

## Summary

Prototype pattern enables:
- Fast object creation by cloning
- Avoiding expensive initialization
- Runtime object configuration
- Reduced need for subclassing

Perfect for: Games, templates, configurations, testing.

**Key Insight**: Create new objects by copying existing ones instead of instantiating from scratch, saving initialization costs!